In [1]:
# k crossover d，close > ema 8, ema 8 > ema 18, ema 18 > 38, take profit atr, stop loss atr
# Variables :
# time - 15m, 1h
# start time - 1609492611000, 1641028611000, 1672564611000 (2021, 2022, 2023)
# tp atr - 4, 6
# sl atr - 3, 5
# sl - Close, Low

In [2]:
# ! conda install -c conda-forge ta --yes

In [3]:
import requests
import pandas as pd
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [4]:
timezone = 8
symbol = 'ethusdt'
interval = '1h'

# start epoch till now
start_time = 1671366861000
end_time = round(time.time() * 1000)
# end_time = 1672556207000

# step between timestamps in milliseconds
step = 60000 * 3600

In [5]:
dataframes = {}
# 先抓歷史資料好讓技術指標能成型
def create_raw(symbol, interval_arr, start_time, end_time, step):
    
    url = "https://api.binance.com/api/v3/klines"
    
    for interval in interval_arr:

        raw_df = pd.DataFrame()
        
        for timestamp in range(start_time, end_time, step):
            params = {"symbol": symbol.upper(),
                      "interval": interval,
                      "startTime": timestamp,
                      "endTime": timestamp + step}
            response = requests.get(url, params=params).json()
            out = pd.DataFrame(response, columns = ["Open_Time", "Open", "High", "Low", "Close",
                                                   "Volume", "Close_Time", "Quote asset volume",
                                                   "Number of trades", "Taker buy base asset volume",
                                                   "Taker buy quote asset volume", "Ignore"])
            raw_df = pd.concat([raw_df, out], axis = 0)

        raw_df = raw_df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']]

        dataframes[f'df_{interval}'] = raw_df

In [6]:
interval_arr = ['1h', '4h']
start_time_arr = [1663798299000]
ema_arr = [8, 18, 38]

In [7]:
sl_atr = 3
tp_atr = 10
rsi_int = 14
kd_dir = 'Up'
rsi_high = 70

In [8]:
def get_klines(symbol, interval, start_time, end_time):
    global dataframes
    df = dataframes[f'df_{interval}']
    df = df[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)
    df = df.set_index('Open_Time')

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
    df = df[~df.index.duplicated(keep='first')]

    return df

In [9]:
create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)

sample = dataframes[f'df_{interval_arr[0]}']
sample = sample[['Open_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)

sample['Open_Time'] = pd.to_datetime(sample['Open_Time'], unit='ms') + pd.Timedelta(hours=timezone)
sample['Open_Time'] = sample['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
print(sample)

              Open_Time     Open    Close     High      Low      Volume
0   2022-09-22 07:00:00  1243.89  1245.78  1255.36  1239.90  41211.2915
1   2022-09-22 08:00:00  1245.79  1242.88  1252.04  1237.32  35808.9348
2   2022-09-22 09:00:00  1242.87  1256.93  1257.77  1241.44  31181.7227
3   2022-09-22 10:00:00  1256.93  1265.66  1269.00  1252.57  33297.7773
4   2022-09-22 11:00:00  1265.66  1262.91  1271.58  1260.50  21901.1776
..                  ...      ...      ...      ...      ...         ...
26  2023-05-31 09:00:00  1904.97  1903.20  1907.13  1902.40   5934.0383
27  2023-05-31 10:00:00  1903.21  1896.00  1903.21  1884.29  19892.9679
28  2023-05-31 11:00:00  1895.99  1897.71  1897.95  1892.81   5046.6592
29  2023-05-31 12:00:00  1897.71  1874.01  1898.53  1870.00  32418.5007
30  2023-05-31 13:00:00  1874.01  1868.23  1875.62  1861.76  23436.2409

[6030 rows x 6 columns]


In [10]:
df = get_klines(symbol, interval_arr[0], start_time, end_time).copy()
df = df.astype(float).round(2)

for interval in interval_arr:
    raw_df = get_klines(symbol, interval, start_time, end_time)
    # ema
#         for ema in ema_arr:
#             column_name = f'ema_{ema}_{interval}'
#             df[column_name] = ta.trend.EMAIndicator(raw_df.Close, window=ema, fillna=True).ema_indicator()

    # rsi
    rsi = ta.momentum.RSIIndicator(raw_df.Close, window = rsi_int)
    df[f'rsi_{interval}'] = rsi.rsi()

    # atr
#         df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)


# reset index and set current index as a column
df = df.reset_index()

# set new index with integers
df = df.set_index(pd.RangeIndex(len(df)))

# time_format(timezone)
df['Open_Time'] = df['Open_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')

# fill up higher time frame empty values with equal interval between each value
df = df.replace('', np.nan)
#     df['ema_8_4h'] = df['ema_8_4h'].interpolate()
#     df['ema_38_4h'] = df['ema_38_4h'].interpolate()
df['rsi_4h'] = df['rsi_4h'].interpolate()

In [11]:
def check_cross(df, kd_dir):
    up = df['slow_k'] > df['slow_d']
    down = df['slow_k'] < df['slow_d']
    if kd_dir == 'Up':
        return up.diff() & up
    if kd_dir == 'Any':
        return up.diff()
    if kd_dir == 'Down':
        return down.diff() & down


def indicators(df, kd_dir):

# ema
    for i in (8, 18, 38):
        df['ema_'+str(i)] = ta.trend.ema_indicator(df.Close, window=i)

# bband
#     bb_int = 30
#     bb_dev = 2
#     bb = ta.volatility.BollingerBands(df['Close'], window=bb_int, window_dev=bb_dev)
#     df['bb_u'] = bb.bollinger_hband()
#     df['bb_m'] = bb.bollinger_mavg()
#     df['bb_l'] = bb.bollinger_lband()        
        
# atr
    df['atr'] = ta.volatility.average_true_range(df.High, df.Low, df.Close)
    
# rsi
#     rsi_int = 14
#     df['rsi'] = ta.momentum.RSIIndicator(df['Close'], window = rsi_int).rsi()

# kd
    df['slow_k']= ta.momentum.stoch(df['High'], df['Low'], df['Close'], 17, 5)
    df['slow_d'] = ta.momentum.stoch_signal(df['High'], df['Low'], df['Close'], 17, 5)
    
# kd cross
    df['kd_cross'] = check_cross(df, kd_dir)

In [12]:
indicators(df, kd_dir)
print(df)

                Open_Time     Open    Close     High      Low    Volume  \
0     2022-09-22 07:00:00  1243.89  1245.78  1255.36  1239.90  41211.29   
1     2022-09-22 08:00:00  1245.79  1242.88  1252.04  1237.32  35808.93   
2     2022-09-22 09:00:00  1242.87  1256.93  1257.77  1241.44  31181.72   
3     2022-09-22 10:00:00  1256.93  1265.66  1269.00  1252.57  33297.78   
4     2022-09-22 11:00:00  1265.66  1262.91  1271.58  1260.50  21901.18   
...                   ...      ...      ...      ...      ...       ...   
6025  2023-05-31 09:00:00  1904.97  1903.20  1907.13  1902.40   5934.04   
6026  2023-05-31 10:00:00  1903.21  1896.00  1903.21  1884.29  19892.97   
6027  2023-05-31 11:00:00  1895.99  1897.71  1897.95  1892.81   5046.66   
6028  2023-05-31 12:00:00  1897.71  1874.01  1898.53  1870.00  32418.50   
6029  2023-05-31 13:00:00  1874.01  1868.23  1875.62  1861.76  23436.24   

         rsi_1h     rsi_4h        ema_8       ema_18       ema_38        atr  \
0           NaN    

In [13]:
def conditions(df):
    # k crossover d，close > ema 8, ema 8 > ema 14, ema 14 > 50, take profit 2 atr, stop loss 3 atr

    df['c5'] = False
    for i, row in df.iterrows():
        # c1
        df['c1'] = df['kd_cross']
        # c2
        df['c2'] = df['Close'] >= df['ema_8']
        # c3
        df['c3'] = df['ema_8'] >= df['ema_18']
        # c4
        df['c4'] = df['ema_18'] >= df['ema_38']
        # c5
        
        if i >= 14:
            if (df.loc[i-14:i-1, 'rsi_4h'] <= rsi_high).any():
                df.loc[i, 'c5'] = True

    # 條件達成
    df['signal'] = False
    df.loc[df.c1 & df.c2 & df.c3 & df.c4 & df.c5, 'signal'] = True


    # 下一根進場
    df['openbuy'] = False
    for i in range(len(df) - 1):
        if df.loc[i, 'signal'] == True:
            df.loc[i + 1, 'openbuy'] = True
#     print(df[df['Close_Time'].str.contains('2023-03-11 16')])
    
    return df

In [14]:
conditions(df)

,Open_Time,Open,Close,High,Low,Volume,rsi_1h,rsi_4h,ema_8,ema_18,...,slow_k,slow_d,kd_cross,c5,c1,c2,c3,c4,signal,openbuy
0,2022-09-22 07:00:00,1243.89,1245.78,1255.36,1239.90,41211.29,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
1,2022-09-22 08:00:00,1245.79,1242.88,1252.04,1237.32,35808.93,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
2,2022-09-22 09:00:00,1242.87,1256.93,1257.77,1241.44,31181.72,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
3,2022-09-22 10:00:00,1256.93,1265.66,1269.00,1252.57,33297.78,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
4,2022-09-22 11:00:00,1265.66,1262.91,1271.58,1260.50,21901.18,NaN,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025,2023-05-31 09:00:00,1904.97,1903.20,1907.13,1902.40,5934.04,51.929502,57.222358,1903.907857,1903.277126,...,46.240748,48.375536,False,True,False,False,True,True,False,True
6026,2023-05-31 10:00:00,1903.21,1896.00,1903.21,1884.29,19892.97,44.534268,53.910208,1902.150555,1902.511113,...,35.799450,44.425180,False,True,False,False,False,True,False,False
6027,2023-05-31 11:00:00,1895.99,1897.71,1897.95,1892.81,5046.66,46.483545,50.598058,1901.163765,1902.005732,...,42.361111,42.714309,False,True,False,False,False,True,False,False
6028,2023-05-31 12:00:00,1897.71,1874.01,1898.53,1870.00,32418.50,30.490042,47.285908,1895.129595,1899.058813,...,8.774617,37.254585,False,True,False,False,False,True,False,False


In [15]:
entry_df = conditions(df).copy()

in_position = False
stop_loss = np.nan
take_profit = np.nan
close_val = entry_df['Close']
atr_val = entry_df['atr']


for index, row in entry_df.iterrows():

    if index == 0:
        continue
    
    elif entry_df.at[index, 'openbuy'] == True:
   
        entry_df.at[index, 'entry_p'] = close_val.shift(1).at[index]
        entry_df.at[index, 'stop_loss'] = close_val.shift(1).at[index] - sl_atr * atr_val.shift(1).at[index]
        entry_df.at[index, 'take_profit'] = close_val.shift(1).at[index] + tp_atr * atr_val.shift(1).at[index]
        entry_df.at[index, 'position'] = 'Buy'
        in_position = True
        stop_loss = entry_df.at[index, 'stop_loss']
        take_profit = entry_df.at[index, 'take_profit']
    

    # 吃筍
    #-----------------------------重要-----------------------------
    # 若用 if 寫，則有可能入場馬上吃筍，若用 elif 則一個 iteration 只會執行一次
    elif in_position == True and (entry_df.at[index, 'Close'] <= stop_loss):
        entry_df.at[index, 'position'] = 'Stop'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan

    # set take profit
    elif in_position == True and (entry_df.at[index, 'High'] >= take_profit):
        entry_df.at[index, 'position'] = 'Sell'
        in_position = False
        stop_loss = np.nan
        take_profit = np.nan
        
entry_df.to_csv('entry_df.csv')
        
raw_entry_df = entry_df.copy()

# 過濾有訊號或事件發生的Ｋ線
entry_df = entry_df[(entry_df['openbuy'] == True) |
              (entry_df['signal'] == True) | 
              (entry_df['position'] == 'Buy') |
              (entry_df['position'] == 'Sell') |
              (entry_df['position'] == 'Stop')]

print(entry_df)

                Open_Time     Open    Close     High      Low    Volume  \
57    2022-09-24 16:00:00  1322.40  1329.24  1331.15  1317.03  23193.98   
58    2022-09-24 17:00:00  1329.24  1326.59  1329.99  1321.33  24322.72   
60    2022-09-24 19:00:00  1326.00  1326.99  1332.74  1323.81  25251.25   
61    2022-09-24 20:00:00  1326.99  1331.99  1336.39  1326.15  24239.12   
67    2022-09-25 02:00:00  1336.36  1343.69  1344.00  1333.85  18576.62   
...                   ...      ...      ...      ...      ...       ...   
6021  2023-05-31 05:00:00  1903.72  1905.59  1914.78  1902.43   9657.80   
6022  2023-05-31 06:00:00  1905.60  1904.40  1905.97  1898.96   5258.59   
6024  2023-05-31 08:00:00  1900.58  1904.96  1905.27  1898.14   5532.64   
6025  2023-05-31 09:00:00  1904.97  1903.20  1907.13  1902.40   5934.04   
6028  2023-05-31 12:00:00  1897.71  1874.01  1898.53  1870.00  32418.50   

         rsi_1h     rsi_4h        ema_8       ema_18  ...     c1     c2  \
57    57.478921  61.6269

In [17]:
# 部位回測

pos_df = entry_df.copy()
pos_df = pos_df.reset_index(drop = True)
pos_df = pos_df[(pos_df['position'] == 'Buy') |
              (pos_df['position'] == 'Sell') |
              (pos_df['position'] == 'Stop')]

# 一次進場多少單位
pos_size = 1

col = ['Open_Time', 'Open', 'Close', 'High', 'Low', 'ema_8', 'ema_18', 'ema_38', 'atr', 'kd_cross', 'position','entry_p', 'stop_loss', 'take_profit']
pos = pos_df[col]
pos = pos.reset_index(drop = True)


for index, row in pos.iterrows():
    
    current_pos = 0
    
    # 進場
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'size'] = pos_size
        pos.exit_p = np.nan
    
    # 出場
    if pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        
        #-----------------------------重要-----------------------------
        # 實戰需即刻出場
        
        # 停利：達成條件時收盤價
        if pos.at[index, 'position'] == 'Sell':
#             pos.at[index, 'exit_p'] = pos.at[index, 'Close']
            for i in range(index -1, -1, -1):
                if pos.at[i, 'position'] == 'Buy':
                    pos.at[index, 'exit_p'] = pos.at[i, 'take_profit']
                break

        # 停損：打到進場停損點（往回跌代，直到最近的'Buy'及其'stop_loss'）
        if pos.at[index, 'position'] == 'Stop':
            pos.at[index, 'exit_p'] = pos.at[index, 'Open']
#             for i in range(index -1, -1, -1):
#                 if pos.at[i, 'position'] == 'Buy':
#                     pos.at[index, 'exit_p'] = pos.at[i, 'stop_loss']
#                 break

        # 計算每次出場部位大小（每次出場皆清倉）
        for i in range(index -1, -1, -1):
            if pos.at[i, 'position'] == 'Buy':
                current_pos += pos.at[i, 'size']
                if i == 0:
                    pos.at[index, 'size'] = -current_pos
                else:
                    continue
            else:
                pos.at[index, 'size'] = -current_pos
                current_pos = 0
                break


# 計算部位價值
for index, row in pos.iterrows():
    if pos.at[index, 'position'] == 'Buy':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'entry_p'], 4)
    elif pos.at[index, 'position'] == 'Sell' or pos.at[index, 'position'] == 'Stop':
        pos.at[index, 'amt'] = round(pos.at[index, 'size'] * pos.at[index, 'exit_p'], 4)


# 若最後一筆為 Buy，移除該單，迭代驗證
for index, row in pos.iloc[::-1].iterrows():
    if row['position'] == 'Buy':
        pos = pos.drop(index)
    else:
        break


# 手續費、滑點、價差
fee = 0.05 / 100
amt_abs_sum = pos.amt.abs().sum()
ttl_fee = amt_abs_sum * fee


# 損益
leverage = 10
ttl_profit = -pos.amt.sum() - ttl_fee


# 計算獲利/虧損次數
agg_amts = []

for i in range(len(pos) - 1, -1, -1):

    if pos.loc[i, 'position'] in ['Stop', 'Sell']:
        
        # look out for the + sign
        total_amt = pos.loc[i, 'amt'] + np.absolute(pos.loc[i, 'amt']) * fee
        
        # iterate backwards from the current row until reaching another 'Stop' or 'Sell'
        # watch out for the + in total_amt += trading_fee
        j = i - 1
        while j >= 0 and pos.loc[j, 'position'] not in ['Stop', 'Sell']:
            total_amt += pos.loc[j, 'amt']
            trading_fee = np.absolute(pos.loc[j, 'amt']) * fee
            total_amt += trading_fee
            j -= 1
        
        # add the aggregated amount to the list
        agg_amts.append(total_amt)

agg_amts.reverse()


# 計算進場最大部位，最大損益
consec_entry = 0
position_amt_sum = 0
max_consec_entry = 0
max_position = 0
max_profit = 0
max_loss = 0

for index, row in pos.iterrows():
    
    if row['position'] == 'Buy':
        
        consec_entry += 1
        position_amt_sum += row['amt']
        
    elif row['position'] in ['Sell', 'Stop']:
        
        if consec_entry > max_consec_entry:
            max_consec_entry = consec_entry
            max_position = position_amt_sum
            
        position_amt_sum += row['amt']
        
        if -position_amt_sum > max_profit:
            max_profit = -position_amt_sum
            
        if -position_amt_sum < max_loss:
            max_loss = -position_amt_sum
            
        consec_entry = 0
        position_amt_sum = 0
    
    else:
        pass

# 最大部位
profit_per = "{:.2f}%".format(ttl_profit / (max_position/leverage) * 100)


# 勝率
wins = 0
loses = 0

for trade in agg_amts:
    if trade < 0:
        wins += 1
    elif trade > 0:
        loses += 1

win_rate = "{:.2f}%".format(wins / (wins + loses) * 100)


# 結果
result = {'Profit': [round(ttl_profit, 2)],
          'Fee': [round(ttl_fee, 2)],
          'Max_Profit': [round(max_profit, 2)],
          'Max_Loss': [round(max_loss, 2)],
          'Max_Entry': [max_consec_entry],
          'Max_Position': [round(max_position, 2)],
          'Profit_%': [profit_per],
          'Win_Rate': [win_rate]}


result_df = pd.DataFrame(result)

print(result_df)
print()
print(pos)

pos.to_csv('pos.csv')


    Profit     Fee  Max_Profit  Max_Loss  Max_Entry  Max_Position Profit_%  \
0  3293.76  494.01     1008.68   -348.06         17      20347.97  161.87%   

  Win_Rate  
0   30.16%  

               Open_Time     Open    Close     High      Low        ema_8  \
0    2022-09-24 17:00:00  1329.24  1326.59  1329.99  1321.33  1324.926792   
1    2022-09-24 20:00:00  1326.99  1331.99  1336.39  1326.15  1326.995925   
2    2022-09-25 03:00:00  1343.70  1342.31  1348.35  1339.59  1338.462084   
3    2022-09-26 01:00:00  1303.54  1297.38  1304.39  1281.91  1310.099279   
4    2022-09-27 03:00:00  1330.95  1330.38  1340.42  1326.85  1323.656806   
..                   ...      ...      ...      ...      ...          ...   
372  2023-05-31 01:00:00  1904.23  1902.80  1912.64  1897.61  1903.309726   
373  2023-05-31 03:00:00  1908.01  1907.32  1908.40  1905.00  1905.015020   
374  2023-05-31 06:00:00  1905.60  1904.40  1905.97  1898.96  1904.803637   
375  2023-05-31 09:00:00  1904.97  1903.20  19

In [ ]:
cumulative_values = []
cumulative_sum = 0
for value in agg_amts:
    cumulative_sum -= value
    cumulative_values.append(cumulative_sum)

plt.fill_between(range(len(cumulative_values)), cumulative_values, 0)

plt.xlabel('Index')
plt.ylabel('Cumulative Sum')
plt.title('Cumulative Sum of Values')

plt.show()

print(cumulative_values)

In [ ]:
plt_df = raw_entry_df.copy()

plt.figure(figsize=(80, 36))
plt.ylabel('Close')
plt.plot(plt_df.Close, label = 'Close', c ='black')

i = 0
while i < len(plt_df):
    if pd.isna(plt_df.position[i]):
        i += 1
    elif plt_df.position[i] == 'Buy':
        j = i + 1
        while j < len(plt_df) and (pd.isna(plt_df.position[j]) or plt_df.position[j] == 'Buy'):
            j += 1
        if j < len(plt_df) and plt_df.position[j] == 'Sell':
            # Paint all previous 'Buy's green dot
            for k in range(i, j+1):
                plt.plot(k, plt_df.entry_p[k], 'go', label='_', markersize=20)
            i = j
        elif j < len(plt_df) and plt_df.position[j] == 'Stop':
            # Paint all previous 'Buy's red dot
            for k in range(i, j+1):
                plt.plot(k, plt_df.entry_p[k], 'ro', label='_', markersize=20)
            i = j + 1
        else:
            i = j
    else:
        i += 1

plt.legend()
plt.show()


# 之所以出現大量紅點，原因為：連續加倉會調整整個部位停利和停損，因此一些進倉部位會因為調高後的＂整個部位＂停損而出場，但該入場損益依舊為正
# 意思是該部位之停損已經高於許多前面部位之進場點位